In [75]:
import spacy
from spaczz.matcher import FuzzyMatcher
import pandas as pd
import Levenshtein
import re

In [87]:
sub_data = pd.read_csv("C:/Python/Substanzen/Test_Daten.csv", sep = ";", encoding="utf-8")
sub_data["Bezeichnung"] = sub_data["Bezeichnung"].replace({pd.NA: 'NA', '': 'NA'})
sub_data["ID"] = range(1, len(sub_data) + 1)
ref_tab = pd.read_csv("C:/Python/Substanzen/substanz.csv", sep = ";", encoding="utf-8")
sub_data.head()

,Bezeichnung,ID
0,Cisplatin,1
1,Etoposid,2
2,Carboplatin,3
3,Calciumfolinat,4
4,Avelumab,5


In [88]:
nlp = spacy.blank("en")

matcher = FuzzyMatcher(nlp.vocab)
matcher.add("Substance", [nlp(str(sub)) for sub in ref_tab["Substanz"]])

results = []
for _, row in sub_data.iterrows():
    text = row["Bezeichnung"]
    id_num = row["ID"]

    doc = nlp(str(text))
    matches = matcher(doc)

    match_found = False  # Track whether we found a good match

    for match_id, start, end, ratio, pattern in matches:
        if ratio > 80:
            results.append({
                "ID": id_num,
                "input": text,
                "match": doc[start:end].text,
                "matched_to": pattern,
                "similarity": ratio
            })
            match_found = True

    # If no match was found, add empty row for this ID
    if not match_found:
        results.append({
            "ID": id_num,
            "input": text,
            "match": "",
            "matched_to": "",
            "similarity": ""
        })

# Now create the results DataFrame
results_df = pd.DataFrame(results)


In [89]:
missing_id_list = sub_data.loc[~sub_data["ID"].isin(results_df["ID"]), "ID"].unique().tolist()
print("IDs in df1 but not in df2:", missing_id_list)

IDs in df1 but not in df2: []


In [90]:
pattern = r"[,+;\\+]|\bund\b"  # cleaner pattern
filtered_df = results_df[~results_df['input'].str.contains(pattern, case=False, regex=True, na=False)].copy()
filtered_df.head()

,ID,input,match,matched_to,similarity
0,1,Cisplatin,Cisplatin,Cisplatin,100
1,2,Etoposid,Etoposid,Etoposid,100
2,3,Carboplatin,Carboplatin,Carboplatin,100
3,4,Calciumfolinat,Calciumfolinat,Calciumlevofolinat,88
4,5,Avelumab,Avelumab,Avelumab,100


In [91]:
filtered_df.loc[:,'match_count'] = filtered_df.groupby('ID')['ID'].transform('count')
subset_df = filtered_df[filtered_df['match_count'] > 1].sort_values(by = "ID", ascending = True)
subset_df['LV_distance'] = subset_df.apply(lambda row: Levenshtein.distance(str(row['input']), str(row['match'])), axis=1)
best_matches = subset_df.loc[subset_df.groupby('ID')['LV_distance'].idxmin()].reset_index(drop=True).sort_values(by = "ID", ascending = True)
df_to_merge_best_match = best_matches.drop(columns = ["match_count","LV_distance"])
df_to_merge_best_match.head()

,ID,input,match,matched_to,similarity
0,10,Paclitaxel nab,Paclitaxel nab,Paclitaxel nab,100
1,11,Epirubicin,Epirubicin,Pirarubicin,86
2,18,Rituximab,Rituximab,Isatuximab,84
3,31,Prednison,Prednison,Prednisolon,90
4,32,Daunorubicin,Daunorubicin,Idarubicin,82


In [92]:
#filter out IDs from best_matches
ID_with_best_matches = best_matches["ID"].unique()
results_df_for_collapse = results_df[~results_df['ID'].isin(ID_with_best_matches)].reset_index(drop=True)

#collapse all other rows
collapsed_df = (
    results_df_for_collapse.groupby("ID").agg({
        "input": "first",
        "match": lambda x: "; ".join(x.dropna().astype(str)),
        "matched_to": lambda x: "; ".join(dict.fromkeys(x.dropna().astype(str))),
        "similarity": lambda x: "; ".join(dict.fromkeys(x.dropna().astype(str))),
    }).reset_index()
)
#merge both DFs
combined_df = pd.concat([df_to_merge_best_match, collapsed_df], ignore_index=True, sort=False).sort_values(by = "ID", ascending = True)
#check for completness
print(len(combined_df))
print(len(sub_data))

missing_ids = sub_data[~sub_data["ID"].isin(combined_df["ID"])]
print(missing_ids)

merged_df = pd.merge(sub_data, combined_df, on="ID", how="left")
rows_with_na = merged_df[merged_df["input"].isna()]

# Display the rows that have NaN in the merged columns
print(rows_with_na)
merged_df.to_csv("output.csv", sep= ";")

4268
4268
Empty DataFrame
Columns: [Bezeichnung, ID]
Index: []
Empty DataFrame
Columns: [Bezeichnung, ID, input, match, matched_to, similarity]
Index: []
